# So easy, *voilà* IPyMaps!

In this example notebook, we demonstrate how voila can render custom interactive maps with built-in search. Try entering something like the following into the search field:

- 52.5 13.4
- Paris
- https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json
- berlin_stops (a predefined GeoJSON object)

In [ ]:
import re
import os

import requests
import geojson
from ipywidgets import Text, HTML, Layout
from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl, LayersControl, ZoomControl, basemaps
from geopy.geocoders import Nominatim

In [ ]:
def bbox(coord_list):
    "Find bounding box (lower-left and upper-right points) for a list of coordinates."

    box = []
    for i in (0, 1):
        res = sorted(coord_list, key=lambda x:x[i])
        box.append((res[0][i], res[-1][i]))
    ll, ur = [box[0][0], box[1][0]], [box[0][1], box[1][1]]
    return [ll, ur]

In [ ]:
class SearchController:
    """
    A controller acting as a relay between a search field and a map.
    """
    def __init__(self, a_map, a_text, geocoder=None, globs=None):
        self.geocoder = geocoder or Nominatim(user_agent="ipymaps")
        self.globs = globs or {}
        self.a_map = a_map
        self.a_text = a_text
        self.a_text.on_submit(self.text_changed)
        
    def text_changed(self, widget):
        "This is called whenever the user hits enter in the search text field."

        res = self.run_query(widget.value)
        if type(res) == tuple:
            self.a_map.center = res
        elif hasattr(res, '__geo_interface__'):
            data = res
            # jump to center of data
            ll, ur = bbox(list(geojson.utils.coords(data)))
            center = [(ur[0]+ll[0])/2, (ur[1]+ll[1])/2]
            self.a_map.center = list(reversed(center))
            name = os.path.basename(widget.value)
            self.a_map.add_layer(GeoJSON(data=data, name=name))
            
    def add_geojson(self, url):
        "Load a GeoJSON string from the given URL, local or remote."

        # load data
        if url.startswith('file://'):
            gj = open(url[7:]).read()
        else:
            gj = requests.get(url).content
        return geojson.loads(gj)
        
    def run_query(self, text):
        "Run a query as given by the ``text`` string."
        
        text = text.strip()

        # GeoJSON objects in passed namespace        
        if text in self.globs:
            obj = self.globs[text]
            if hasattr(obj, '__geo_interface__'):
                return obj
        
        # lat lon
        try:
            values = re.split('[/,;: ]', text)
            lat, lon = [v for v in values if v not in '/,;:']
            return tuple(list(map(float, (lat, lon))))
        except ValueError:
            pass

        # URL pointing to some GeoJSON string
        url = text
        if url.endswith('.geojson') or url.endswith('.geo.json'):
            return self.add_geojson(url)

        # address to run geolocation on
        loc = self.geocoder.geocode(text)
        if loc:
            center = loc.latitude, loc.longitude
            return center

In [ ]:
# provide a default identifer that can be used in the demo UI
url = 'https://gist.githubusercontent.com/deeplook/40ae4f4ecd8ec8d4be51/raw/c1dcdf253e47faf8586cffa4e009a705c8c0906b/stops_berlin.geojson'
berlin_stops = geojson.loads(requests.get(url).content)

In [ ]:
bm = basemaps.OpenStreetMap['Mapnik']
m = Map(center=[50, 18], zoom=4,
        zoom_control=False,
        basemap=bm,
        layout=Layout(height='600px'))

m.layers[0].name = bm['name']

# setup search text field and controller
search_tx = Text('')
wc = WidgetControl(widget=search_tx, position='topleft')
con = SearchController(m, search_tx, globs=globals())

# add controls
m.add_control(wc)
m.add_control(ZoomControl(position='topleft'))
m.add_control(FullScreenControl(position='topleft'))
m.add_control(LayersControl(position='topright'))

m